In [1]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import date
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from lxml import etree
import matplotlib.dates as mdates
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error, median_absolute_error, explained_variance_score
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from matplotlib.dates import AutoDateLocator, AutoDateFormatter
from scipy.stats import randint
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.preprocessing.sequence import TimeseriesGenerator
from keras import layers, models, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM
import graphviz
from IPython.display import Image
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from keras.metrics import MeanSquaredError, MeanAbsoluteError

In [ ]:
import pandas as pd

# Load weather data
weather_df = pd.read_csv('weather.csv', parse_dates=['date'])

# Convert weather data timestamps to match house timestamps format (if necessary)
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%d/%m/%Y %H:%M')  # Assuming weather data format is 'dd/mm/yyyy hh:mm'

# Dictionary to hold the houses dataframes
houses = {}
house_names = []

# Loop through house files, skipping number 14
for i in range(1, 21):
    file_name = f'H{i}_W.csv'
    
    try:
        # Load house data
        house_df = pd.read_csv(f'{file_name}', parse_dates=['date'])
        
        # Merge the house data with weather data on the 'date' column (inner join to keep only matching timestamps)
        merged_df = pd.merge(house_df, weather_df, on='date', how='inner')
        
        # Add the merged dataframe to the dictionary
        houses[file_name] = merged_df
        house_names.append(file_name)
        
        # Display the first few rows of the merged dataframe to verify
        print(f"Merged data for {file_name}:")
        print(houses[file_name].head())
        
    except FileNotFoundError:
        print(f'File not found: {file_name}')
        continue


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

for name, df in houses.items():
    print(f"House: {name}")
    
    # Strip leading/trailing spaces from column names
    df.columns = df.columns.str.strip()

    # Print cleaned columns to verify
    print("Cleaned Columns in the dataset:")
    print(df.columns)  # Print the cleaned column names to check for discrepancies

    # Define the energy-related columns (ensure these match the actual column names in your data)
    energy_columns = ['Discharge(W)', 'Charge(W)', 'Production(W)', 'Consumption(W)', 'State of Charge(%)']
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

    # Check if each energy column exists in the dataset before processing
    for column in energy_columns:
        if column in df.columns:
            print(f"Processing {column} for {name}...")

            # Replace 0 values in the column with a small value (1e-6)
            df[column] = df[column].replace(0, 1e-6)
            # Verify that there are no more 0 values in the dataset
            zero_values_after_replacement = (df[column] == 0).sum()
            print(f"Zero values after replacement for {column}: {zero_values_after_replacement}")
    
            # Fill missing values for the column
            df[column] = df[column].fillna((df[column].ffill() + df[column].bfill()) / 2)

            # Use ffill() and bfill() directly
            df[column] = df[column].bfill()
            df[column] = df[column].ffill()



            # Plot the energy columns
            plt.figure(figsize=(20, 10))
            plt.plot(df[column], alpha=0.7)  # Set opacity for better visualization
            plt.title(f'{column} Over Time for {name}')
            plt.ylabel('Energy (W)' if 'Charge' in column or 'Discharge' in column or 'Production' in column or 'Consumption' in column else 'State of Charge (%)')
            plt.xlabel('Time')

            # Formatting date on x-axis
            plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # Set major ticks to each month
            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Format ticks as Year-Month
            plt.xticks(rotation=45)  # Rotate labels to 45 degrees for better legibility
            
            plt.legend([column])
            plt.grid(True)
            plt.tight_layout()  # Adjust layout to prevent clipping of tick labels
            
            if column in ['Production', 'Consumption']:
                plt.savefig(f'{name}_{column}.jpeg', format='jpeg', dpi=300)
            plt.show()
        else:
            print(f"Column {column} not found in {name}")
            


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer  # For handling NaNs


for name, df in houses.items():
    print(name)
    df = df.apply(pd.to_numeric, errors='coerce')
        # Handle missing values by imputing with mean
    # Replace NaN values with 0
    df.fillna(0, inplace=True)
    # Define features and targets
    X = df.drop(['Consumption(W)', 'Production(W)'], axis=1)
    y_consumption = df['Consumption(W)']
    y_production = df['Production(W)']

    # Splitting data into train and test sets
    X_train, X_test, y_train_consumption, y_test_consumption = train_test_split(X, y_consumption, test_size=0.2, random_state=42)
    X_train, X_test, y_train_production, y_test_production = train_test_split(X, y_production, test_size=0.2, random_state=42)

    ### 1. Separate RandomForestRegressor for Consumption
    regressor_consumption = RandomForestRegressor(n_estimators=20, max_depth=1, max_features='sqrt', n_jobs=-1, random_state=42, warm_start=True)
    regressor_consumption.fit(X_train, y_train_consumption)

    rmse_consumption = mean_squared_error(y_test_consumption, predictions_consumption, squared=False)
    mae_consumption = mean_absolute_error(y_test_consumption, predictions_consumption)
    mse_consumption = mean_squared_error(y_test_consumption, predictions_consumption)
    r2_consumption = r2_score(y_test_consumption, predictions_consumption)
    mape_consumption = mean_absolute_percentage_error(y_test_consumption, predictions_consumption)

    print(f'RandomForestRegressor Evaluation Metrics for Consumption ({name}):')
    print(f'RMSE: {rmse_consumption}')
    print(f'MAE: {mae_consumption}')
    print(f'MSE: {mse_consumption}')
    print(f'R²: {r2_consumption}')
    print(f'MAPE: {mape_consumption}\n')

    ### 2. Separate RandomForestRegressor for Production
    regressor_production = RandomForestRegressor(n_estimators=20, max_depth=2, max_features='sqrt',n_jobs=-1, random_state=42, warm_start=True)
    regressor_production.fit(X_train, y_train_production)

    predictions_production = regressor_production.predict(X_test)
    predictions_production += np.random.normal(0, 0.3, predictions_production.shape)

    rmse_production = mean_squared_error(y_test_production, predictions_production, squared=False)
    mae_production = mean_absolute_error(y_test_production, predictions_production)
    mse_production = mean_squared_error(y_test_production, predictions_production)
    r2_production = r2_score(y_test_production, predictions_production)
    mape_production = mean_absolute_percentage_error(y_test_production, predictions_production)

    print(f'RandomForestRegressor Evaluation Metrics for Production ({name}):')
    print(f'RMSE: {rmse_production}')
    print(f'MAE: {mae_production}')
    print(f'MSE: {mse_production}')
    print(f'R²: {r2_production}')
    print(f'MAPE: {mape_production}\n')

    y = df[['Consumption(W)', 'Production(W)']]  

    # Splitting data into train and test sets for MultiOutput
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    regressor = RandomForestRegressor(n_estimators=20, max_depth=2, max_features='sqrt', n_jobs=-1, random_state=42, warm_start=True)
    multioutput_regressor = MultiOutputRegressor(regressor)
    multioutput_regressor.fit(X_train, y_train)

    rmse = mean_squared_error(y_test, predictions, squared=False)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    mape = mean_absolute_percentage_error(y_test, predictions)

    print(f'MultiOutput RandomForestRegressor Evaluation Metrics for both ({name}):')
    print(f'RMSE: {rmse}')
    print(f'MAE: {mae}')
    print(f'MSE: {mse}')
    print(f'R²: {r2}')
    print(f'MAPE: {mape}\n')
    
    gb_regressor_consumption = GradientBoostingRegressor(n_estimators=20,max_depth=5,random_state=42)
    rmse_consumption_gb = mean_squared_error(y_test_consumption, predictions_consumption, squared=False)
    mae_consumption_gb = mean_absolute_error(y_test_consumption, predictions_consumption)
    mse_consumption_gb = mean_squared_error(y_test_consumption, predictions_consumption)
    r2_consumption_gb = r2_score(y_test_consumption, predictions_consumption)
    mape_consumption_gb = mean_absolute_percentage_error(y_test_consumption, predictions_consumption)

    print(f'GradientBoostingRegressor Evaluation Metrics for Consumption ({name}):')
    print(f'RMSE: {rmse_consumption_gb}')
    print(f'MAE: {mae_consumption_gb}')
    print(f'MSE: {mse_consumption_gb}')
    print(f'R²: {r2_consumption_gb}')
    print(f'MAPE: {mape_consumption_gb}\n')

    gb_regressor_production = GradientBoostingRegressor(n_estimators=20,max_depth=5,random_state=42)
    gb_regressor_production.fit(X_train, y_train_production)

    rmse_production_gb = mean_squared_error(y_test_production, predictions_production, squared=False)
    mae_production_gb = mean_absolute_error(y_test_production, predictions_production)
    mse_production_gb = mean_squared_error(y_test_production, predictions_production)
    r2_production_gb = r2_score(y_test_production, predictions_production)
    mape_production_gb = mean_absolute_percentage_error(y_test_production, predictions_production)

    print(f'GradientBoostingRegressor Evaluation Metrics for Production ({name}):')
    print(f'RMSE: {rmse_production_gb}')
    print(f'MAE: {mae_production_gb}')
    print(f'MSE: {mse_production_gb}')
    print(f'R²: {r2_production_gb}')
    print(f'MAPE: {mape_production_gb}\n')


    linear_regressor = LinearRegression(normalize=True)
    multioutput_regressor_linear = MultiOutputRegressor(linear_regressor)
    multioutput_regressor_linear.fit(X_train, y_train)
    
    rmse_linear = mean_squared_error(y_test, predictions, squared=False)
    mae_linear = mean_absolute_error(y_test, predictions)
    mse_linear = mean_squared_error(y_test, predictions)
    r2_linear = r2_score(y_test, predictions)
    mape_linear = mean_absolute_percentage_error(y_test, predictions)

    print(f'Linear Regression (MultiOutput) Evaluation Metrics for both ({name}):')
    print(f'RMSE: {rmse_linear}')
    print(f'MAE: {mae_linear}')
    print(f'MSE: {mse_linear}')
    print(f'R²: {r2_linear}')
    print(f'MAPE: {mape_linear}\n')


In [3]:
# Adapting the concepts of accuracy, precision, recall and confusion matrix by defining thresholds within which predictions are considered "accurate" or "correct." 
# This process is sometimes referred to as "binning" or "categorizing" continuous data to fit a classification model framework. 
def dynamic_regression_accuracy(y_true, y_pred, scale_factor=0.1):
    """Calculate accuracy with a dynamic tolerance based on a scale factor times the standard deviation of y_true."""
    tolerance = scale_factor * np.std(y_true)
    correct = np.abs(y_true - y_pred) <= tolerance
    return np.mean(correct)


def binarize_data(y, threshold):
    """Convert continuous data into binary classes based on a threshold."""
    return np.where(y > threshold, 1, 0)  # 1 for high consumption, 0 for low consumption

def calculate_precision_recall(y_true, y_pred, threshold):
    """Calculate precision for binarized data."""
    y_true_binarized = binarize_data(y_true, threshold)
    y_pred_binarized = binarize_data(y_pred, threshold)
    
    precision = precision_score(y_true_binarized, y_pred_binarized)

    
    return precision

def calculate_recall(y_true, y_pred, threshold):
    """Calculate recall for binarized data."""
    y_true_binarized = binarize_data(y_true, threshold)
    y_pred_binarized = binarize_data(y_pred, threshold)
    recall = recall_score(y_true_binarized, y_pred_binarized)
    return recall

def create_confusion_matrix(y_true, y_pred, threshold):
    """Create a confusion matrix for a given threshold for binarizing continuous outcomes."""
    bins = [0, threshold, np.inf]
    y_true_binned = np.digitize(y_true, bins) - 1  # Convert to 0 or 1
    y_pred_binned = np.digitize(y_pred, bins) - 1
    cm = confusion_matrix(y_true_binned, y_pred_binned)
    return cm

def custom_mape(y_true, y_pred):
    epsilon = 1e-8
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.clip_by_value(tf.cast(y_pred, tf.float32), 0, float('inf'))  # Clip predictions at zero
    
    mape = tf.reduce_mean(tf.abs((y_true - y_pred) / (tf.maximum(tf.abs(y_true), epsilon)))) * 100
    return mape




In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, SimpleRNN, LSTM, Bidirectional, GRU, Conv1D, MaxPooling1D, Flatten, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D, TimeDistributed, Conv2D, Concatenate, Attention
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Dropout, Dense, Input, GlobalAveragePooling1D, Add, Conv1D, MaxPooling1D, Bidirectional, LSTM, Attention, GRU, LeakyReLU, BatchNormalization 
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import optuna
from optuna.integration import TFKerasPruningCallback
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error, explained_variance_score, mean_squared_log_error
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour
from optuna.visualization import plot_parallel_coordinate
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Dropout, Dense, Input, GlobalAveragePooling1D, Add, Conv1D, MaxPooling1D, Bidirectional, GRU, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import RobustScaler


# Define RNN model
def create_rnn_model(input_shape, optimizer='adam', activation = 'relu', learning_rate=0.001):
    inputs = Input(shape=input_shape)
    x = SimpleRNN(32, activation=activation, return_sequences=True)(inputs)
    x = Dropout(0.5)(x)
    x = SimpleRNN(16, activation=activation, return_sequences=False)(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss=tf.losses.Huber(delta=0.5), metrics=['mse', 'mae'])
    return model

# Define LSTM model
def create_lstm_model(input_shape, optimizer='adam', activation = 'relu', learning_rate=0.001):
    inputs = Input(shape=input_shape)
    x = LSTM(32, activation=activation, return_sequences=True)(inputs)
    x = Dropout(0.5)(x)
    x = LSTM(16, activation=activation)(inputs)
    x = Dropout(0.5)(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss=tf.losses.Huber(delta=0.5), metrics=['mse', 'mae'])
    return model

# Define BiLSTM model
def create_bilstm_model(input_shape, optimizer='adam', activation = 'relu', learning_rate=0.001):
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(32, activation=activation))(inputs)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss=tf.losses.Huber(delta=0.5), metrics=['mse', 'mae'])
    return model

# Define GRU model
def create_gru_model(input_shape, optimizer='adam', activation = 'relu', learning_rate=0.001):
    inputs = Input(shape=input_shape)
    x = GRU(32, activation=activation, return_sequences=True)(inputs)
    x = Dropout(0.5)(x)
    x = GRU(16, activation=activation)(inputs)
    x = Dropout(0.5)(x)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss=tf.losses.Huber(delta=0.5), metrics=['mse', 'mae'])
    return model

# Define BiGRU model
def create_bigru_model(input_shape, optimizer='adam', activation = 'relu', learning_rate=0.001):
    inputs = Input(shape=input_shape)
    x = Bidirectional(GRU(32, activation=activation))(inputs)
    outputs = Dense(1)(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=optimizer, loss=tf.losses.Huber(delta=0.5), metrics=['mse', 'mae'])
    return model


# Transformer Encoder Block
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1):
    # Multi-Head Self-Attention
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = Dropout(dropout)(x)
    x = Add()([x, inputs])  # Residual connection
    x = LayerNormalization(epsilon=1e-6)(x)

    # Feed Forward Network
    x_ff = Dense(ff_dim, activation=LeakyReLU())(x)
    x_ff = Dropout(dropout)(x_ff)
    x_ff = Dense(inputs.shape[-1])(x_ff)
    x = Add()([x, x_ff])  # Residual connection
    x = LayerNormalization(epsilon=1e-6)(x)

    return x


# Model Definition
def cnn_lstm_attention_transformer_model(input_shape, num_cnn_filters=16, kernel_size=2, lstm_units=32, gru_units=32, head_size=16, num_heads=2, ff_dim=16, num_transformer_blocks=3, dropout=0.2, weight_decay=1e-6):
    inputs = Input(shape=input_shape)
    
    x = Conv1D(filters=num_cnn_filters, kernel_size=kernel_size, padding='same', activation='relu', kernel_regularizer=l2(weight_decay))(inputs)
    x = MaxPooling1D(pool_size=2)(x) 
    x = Conv1D(filters=num_cnn_filters, kernel_size=kernel_size, padding='same', activation='relu', kernel_regularizer=l2(weight_decay))(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(filters=num_cnn_filters * 2, kernel_size=kernel_size, padding='same', activation='relu', kernel_regularizer=l2(weight_decay))(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(dropout)(x)  # Dropout after pooling
    
    x = LayerNormalization()(x)

    x = Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(weight_decay)))(x)
    x = LeakyReLU()(x)
    x = Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(weight_decay)))(x)
    x = LeakyReLU()(x)
    x = Bidirectional(LSTM(lstm_units // 2, return_sequences=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(weight_decay)))(x)
    x = LeakyReLU()(x)
    x = LayerNormalization()(x)

    x = Bidirectional(GRU(gru_units, return_sequences=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(weight_decay)))(x)
    x = LeakyReLU()(x)
    x = Bidirectional(GRU(gru_units, return_sequences=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(weight_decay)))(x)
    x = LeakyReLU()(x)
    x = Bidirectional(GRU(gru_units // 2, return_sequences=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(weight_decay)))(x)
    x = LeakyReLU()(x)
    x = LayerNormalization()(x)

    # Attention Mechanism (optional)
    attention = Attention()([x, x])  # Self-attention over the LSTM/GRU output
    x = Add()([x, attention])  # Add residual connection after attention
    
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size=head_size, num_heads=num_heads, ff_dim=ff_dim, dropout=dropout)

    x = GlobalAveragePooling1D()(x)
    x = Dropout(dropout)(x)  # Dropout after pooling layer
    outputs = Dense(1, kernel_regularizer=l2(weight_decay))(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss= tf.losses.Huber(delta=1.5), metrics=['mse', 'mae'])
    
    return model

# Function to create lag features
def create_features(df):
    df['Hour'] = df.index.hour
    df['Day_of_Week'] = df.index.dayofweek
    df['Month'] = df.index.month
    # Create lag features for consumption and production
    df['Lag_1'] = df['Consumption(W)'].shift(1)
    df['Lag_60'] = df['Consumption(W)'].shift(60)  # Lag by 1 hour (60 minutes)
    df['Lag_1440'] = df['Consumption(W)'].shift(1440)  # Lag by 1 day (1440 minutes)
    df['Lag_1_prod'] = df['Production(W)'].shift(1)
    df['Lag_60_prod'] = df['Production(W)'].shift(60)
    df['Lag_1440_prod'] = df['Production(W)'].shift(1440)
    df.dropna(inplace=True)
    return df

# Function to create sequences for additional features
def create_sequences_with_additional_features(dataset, additional_features, window_size=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - window_size):
        seq_X = dataset[i:(i + window_size), :]
        seq_features = additional_features[i + window_size - 1]  # Add the corresponding additional feature to each sequence
        dataX.append(np.concatenate((seq_X, np.tile(seq_features, (window_size, 1))), axis=1))
        dataY.append(dataset[i + window_size, :])
    return np.array(dataX), np.array(dataY)

# Function to prepare timeseries generators
def prepare_timeseries_generator(target, features, window_size=24, batch_size=64):
    generator_input = np.concatenate([features, target], axis=1)
    generator = TimeseriesGenerator(generator_input, target, length=window_size, batch_size=batch_size)
    return generator


def print_cpu_usage(phase):
    cpu_percent = psutil.cpu_percent(interval=1)
    print(f"CPU usage during {phase}: {cpu_percent}%")

In [ ]:
RNN_models = {}
for name, df in houses.items():
    print(f"Processing house: {name}")

    df = create_features(df)

    # Convert columns to numeric
    df['Consumption(W)'] = pd.to_numeric(df['Consumption(W)'], errors='coerce')
    df['Production(W)'] = pd.to_numeric(df['Production(W)'], errors='coerce')

    # Scale data
    consumption = df['Consumption(W)'].values.reshape(-1, 1)
    production = df['Production(W)'].values.reshape(-1, 1)
    additional_features = df[['Hour', 'Day_of_Week', 'Month', 'Lag_1', 'Lag_60', 'Lag_1440', 'Lag_1_prod', 'Lag_60_prod', 'Lag_1440_prod', 'speed', 'dir', 'drybulb', 'cbl', 'soltot', 'rain']]
    additional_features = additional_features.apply(pd.to_numeric, errors='coerce')
    additional_features = additional_features.fillna(method='ffill').fillna(method='bfill')
    additional_features = additional_features.values


    split_point = int(len(consumption) * 0.80)

    scaler_consumption = RobustScaler()
    scaler_production = RobustScaler()
    scaler_features = RobustScaler()

    train_consumption = scaler_consumption.fit_transform(consumption[:split_point])
    test_consumption = scaler_consumption.transform(consumption[split_point:])
    train_production = scaler_production.fit_transform(production[:split_point])
    test_production = scaler_production.transform(production[split_point:])
    train_additional_features = scaler_features.fit_transform(additional_features[:split_point])
    test_additional_features = scaler_features.transform(additional_features[split_point:])

    window_size = 60
    batch_size = 64

    # Model for Consumption
    print("Training consumption model...")
    train_generator_consumption = prepare_timeseries_generator(train_consumption, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator_consumption = prepare_timeseries_generator(test_consumption, test_additional_features, window_size=window_size, batch_size=batch_size)

    input_shape = (window_size, train_generator_consumption[0][0].shape[2])
    consumption_model = create_rnn_model(input_shape=input_shape)

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7)

    history_consumption = consumption_model.fit(train_generator_consumption, epochs=100, validation_data=test_generator_consumption, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)

    # Model for Production
    print("Training production model...")
    train_generator_production = prepare_timeseries_generator(train_production, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator_production = prepare_timeseries_generator(test_production, test_additional_features, window_size=window_size, batch_size=batch_size)

    production_model = create_rnn_model_model(input_shape=input_shape)

    history_production = production_model.fit(train_generator_production, epochs=100, validation_data=test_generator_production, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)

    predictions_consumption = consumption_model.predict(test_generator_consumption)
    predictions_production = production_model.predict(test_generator_production)

    actual_consumption = scaler_consumption.inverse_transform(test_consumption[window_size:])
    actual_production = scaler_production.inverse_transform(test_production[window_size:])
    predicted_consumption = scaler_consumption.inverse_transform(predictions_consumption)
    predicted_production = scaler_production.inverse_transform(predictions_production)

    rmse_consumption = mean_squared_error(actual_consumption, predicted_consumption, squared=False)
    mae_consumption = mean_absolute_error(actual_consumption, predicted_consumption)
    r2_consumption = r2_score(actual_consumption, predicted_consumption)
    epsilon = 1e-5
    mape_consumption = np.mean(np.abs((actual_consumption - predictions_consumption) / np.maximum(np.abs(actual_consumption), epsilon))) * 100
    
    print(f"RMSE (Consumption): {rmse_consumption}")
    print(f"MAE (Consumption): {mae_consumption}")
    print(f"R² (Consumption): {r2_consumption}")
    print(f'MAPE (Consumption): {mape_consumption}%')

    rmse_production = mean_squared_error(actual_production, predicted_production, squared=False)
    mae_production = mean_absolute_error(actual_production, predicted_production)
    r2_production = r2_score(actual_production, predicted_production)
    mape_consumption = np.mean(np.abs((actual_production - predicted_production) / np.maximum(np.abs(actual_production), epsilon))) * 100

    print(f"RMSE (Production): {rmse_production}")
    print(f"MAE (Production): {mae_production}")
    print(f"R² (Production): {r2_production}")
    print(f'MAPE (Production): {mape_production}%')
    con_model_weights_path = f"{name}_2RNN_Consumption_weights.h5"
    consumption_model.save_weights(con_model_weights_path)
    prod_model_weights_path = f"{name}_2RNN_Production_weights.h5"
    production_model.save_weights(prod_model_weights_path)
    RNN_models[name] = {
        'model': model,
        'actual_consumption': actual_consumption,
        'predicted_consumption': predicted_consumption,
        'rmse_consumption': rmse_production,
        'mae_consumption': mae_production,
        'r2_consumption': r2_production,
        'mape_consumption':  mape_consumption,
        'actual_production': actual_production,
        'predicted_production': predicted_production,
        'rmse_production': rmse_production,
        'mae_production': mae_production,
        'r2_production': r2_production,
        'mape_production':  mape_production,
        'con_model': consumption_model, 
        'prod_model':production_model,
        'con_model_weights_path': con_model_weights_path,
        'prod_model_weights_path': prod_model_weights_path
    }

    aligned_dates = df.index[split_point + window_size:split_point + window_size + len(actual_consumption)]

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual_consumption, label='Actual Consumption')
    plt.plot(aligned_dates, predicted_consumption, label='Predicted Consumption', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption (W)')
    plt.title(f'Actual vs Predicted Consumption for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2RNN_House_{name}_full_consumption_results.jpeg", format='jpeg', dpi=300)  # Save the full plot for consumption
    plt.show()


    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual_production, label='Actual Production')
    plt.plot(aligned_dates, predicted_production, label='Predicted Production', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Production (W)')
    plt.title(f'Actual vs Predicted Production for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2RNN_House_{name}_full_production_results.jpeg", format='jpeg', dpi=300)  # Save the full plot for production
    plt.show()


    points_in_24h = 1440 
    aligned_dates_24h = aligned_dates[:points_in_24h]

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual_consumption[:points_in_24h], label='Actual Consumption')
    plt.plot(aligned_dates_24h, predicted_consumption[:points_in_24h], label='Predicted Consumption', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption (W)')
    plt.title(f'Actual vs Predicted Consumption for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2RNN_House_{name}_24h_consumption_results.jpeg", format='jpeg', dpi=300)  # Save the 24-hour plot for consumption
    plt.show()

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual_production[:points_in_24h], label='Actual Production')
    plt.plot(aligned_dates_24h, predicted_production[:points_in_24h], label='Predicted Production', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Production (W)')
    plt.title(f'Actual vs Predicted Production for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2RNN_House_{name}_24h_production_results.jpeg", format='jpeg', dpi=300)  # Save the 24-hour plot for production
    plt.show()

    results_df = pd.DataFrame({
        'Date': aligned_dates,
        'Actual_Consumption': actual_consumption.flatten(), 
        'Predicted_Consumption': predicted_consumption.flatten(),
        'Actual_Production': actual_production.flatten(),
        'Predicted_Production': predicted_production.flatten()
    })

    results_df.to_csv(f"2RNN_{name}_results.csv", index=False)
    print(f"Results saved to 2RNN_{name}_results.csv")
    

In [ ]:
LSTM_models = {}
for name, df in houses.items():
    print(f"Processing house: {name}")

    df = create_features(df)

    # Convert columns to numeric
    df['Consumption(W)'] = pd.to_numeric(df['Consumption(W)'], errors='coerce')
    df['Production(W)'] = pd.to_numeric(df['Production(W)'], errors='coerce')

    # Scale data
    consumption = df['Consumption(W)'].values.reshape(-1, 1)
    production = df['Production(W)'].values.reshape(-1, 1)
    additional_features = df[['Hour', 'Day_of_Week', 'Month', 'Lag_1', 'Lag_60', 'Lag_1440', 'Lag_1_prod', 'Lag_60_prod', 'Lag_1440_prod', 'speed', 'dir', 'drybulb', 'cbl', 'soltot', 'rain']]
    additional_features = additional_features.apply(pd.to_numeric, errors='coerce')
    additional_features = additional_features.fillna(method='ffill').fillna(method='bfill')
    additional_features = additional_features.values
    split_point = int(len(consumption) * 0.80)

    scaler_consumption = RobustScaler()
    scaler_production = RobustScaler()
    scaler_features = RobustScaler()

    train_consumption = scaler_consumption.fit_transform(consumption[:split_point])
    test_consumption = scaler_consumption.transform(consumption[split_point:])
    train_production = scaler_production.fit_transform(production[:split_point])
    test_production = scaler_production.transform(production[split_point:])
    train_additional_features = scaler_features.fit_transform(additional_features[:split_point])
    test_additional_features = scaler_features.transform(additional_features[split_point:])

    window_size = 60
    batch_size = 64

    # Model for Consumption
    print("Training consumption model...")
    train_generator_consumption = prepare_timeseries_generator(train_consumption, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator_consumption = prepare_timeseries_generator(test_consumption, test_additional_features, window_size=window_size, batch_size=batch_size)

    input_shape = (window_size, train_generator_consumption[0][0].shape[2])
    consumption_model = cnn_lstm_model(input_shape=input_shape)

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7)

    history_consumption = consumption_model.fit(train_generator_consumption, epochs=100, validation_data=test_generator_consumption, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)

    # Model for Production
    print("Training production model...")
    train_generator_production = prepare_timeseries_generator(train_production, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator_production = prepare_timeseries_generator(test_production, test_additional_features, window_size=window_size, batch_size=batch_size)

    production_model = create_lstm_model_model(input_shape=input_shape)

    history_production = production_model.fit(train_generator_production, epochs=100, validation_data=test_generator_production, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)

    predictions_consumption = consumption_model.predict(test_generator_consumption)
    predictions_production = production_model.predict(test_generator_production)

    actual_consumption = scaler_consumption.inverse_transform(test_consumption[window_size:])
    actual_production = scaler_production.inverse_transform(test_production[window_size:])
    predicted_consumption = scaler_consumption.inverse_transform(predictions_consumption)
    predicted_production = scaler_production.inverse_transform(predictions_production)

    rmse_consumption = mean_squared_error(actual_consumption, predicted_consumption, squared=False)
    mae_consumption = mean_absolute_error(actual_consumption, predicted_consumption)
    r2_consumption = r2_score(actual_consumption, predicted_consumption)
    epsilon = 1e-5
    mape_consumption = np.mean(np.abs((actual_consumption - predictions_consumption) / np.maximum(np.abs(actual_consumption), epsilon))) * 100
    
    print(f"RMSE (Consumption): {rmse_consumption}")
    print(f"MAE (Consumption): {mae_consumption}")
    print(f"R² (Consumption): {r2_consumption}")
    print(f'MAPE (Consumption): {mape_consumption}%')

    rmse_production = mean_squared_error(actual_production, predicted_production, squared=False)
    mae_production = mean_absolute_error(actual_production, predicted_production)
    r2_production = r2_score(actual_production, predicted_production)
    mape_consumption = np.mean(np.abs((actual_production - predicted_production) / np.maximum(np.abs(actual_production), epsilon))) * 100

    print(f"RMSE (Production): {rmse_production}")
    print(f"MAE (Production): {mae_production}")
    print(f"R² (Production): {r2_production}")
    print(f'MAPE (Production): {mape_production}%')
    con_model_weights_path = f"{name}_2LSTM_Consumption_weights.h5"
    consumption_model.save_weights(con_model_weights_path)
    prod_model_weights_path = f"{name}_2LSTM_Production_weights.h5"
    production_model.save_weights(prod_model_weights_path)
    LSTM_models[name] = {
        'model': model,
        'actual_consumption': actual_consumption,
        'predicted_consumption': predicted_consumption,
        'rmse_consumption': rmse_production,
        'mae_consumption': mae_production,
        'r2_consumption': r2_production,
        'mape_consumption':  mape_consumption,
        'actual_production': actual_production,
        'predicted_production': predicted_production,
        'rmse_production': rmse_production,
        'mae_production': mae_production,
        'r2_production': r2_production,
        'mape_production':  mape_production,
        'con_model': consumption_model, 
        'prod_model':production_model,
        'con_model_weights_path': con_model_weights_path,
        'prod_model_weights_path': prod_model_weights_path
    }

    aligned_dates = df.index[split_point + window_size:split_point + window_size + len(actual_consumption)]

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual_consumption, label='Actual Consumption')
    plt.plot(aligned_dates, predicted_consumption, label='Predicted Consumption', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption (W)')
    plt.title(f'Actual vs Predicted Consumption for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2LSTM_House_{name}_full_consumption_results.jpeg", format='jpeg', dpi=300)  # Save the full plot for consumption
    plt.show()


    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual_production, label='Actual Production')
    plt.plot(aligned_dates, predicted_production, label='Predicted Production', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Production (W)')
    plt.title(f'Actual vs Predicted Production for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2LSTM_House_{name}_full_production_results.jpeg", format='jpeg', dpi=300)  # Save the full plot for production
    plt.show()


    points_in_24h = 1440 
    aligned_dates_24h = aligned_dates[:points_in_24h]

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual_consumption[:points_in_24h], label='Actual Consumption')
    plt.plot(aligned_dates_24h, predicted_consumption[:points_in_24h], label='Predicted Consumption', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption (W)')
    plt.title(f'Actual vs Predicted Consumption for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2LSTM_House_{name}_24h_consumption_results.jpeg", format='jpeg', dpi=300)  # Save the 24-hour plot for consumption
    plt.show()

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual_production[:points_in_24h], label='Actual Production')
    plt.plot(aligned_dates_24h, predicted_production[:points_in_24h], label='Predicted Production', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Production (W)')
    plt.title(f'Actual vs Predicted Production for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2LSTM_House_{name}_24h_production_results.jpeg", format='jpeg', dpi=300)  # Save the 24-hour plot for production
    plt.show()

    results_df = pd.DataFrame({
        'Date': aligned_dates,
        'Actual_Consumption': actual_consumption.flatten(), 
        'Predicted_Consumption': predicted_consumption.flatten(),
        'Actual_Production': actual_production.flatten(),
        'Predicted_Production': predicted_production.flatten()
    })

    results_df.to_csv(f"2LSTM_{name}_results.csv", index=False)
    print(f"Results saved to 2LSTM_{name}_results.csv")
    

In [ ]:
BiLSTM_models = {}
for name, df in houses.items():
    print(f"Processing house: {name}")

    df = create_features(df)

    # Convert columns to numeric
    df['Consumption(W)'] = pd.to_numeric(df['Consumption(W)'], errors='coerce')
    df['Production(W)'] = pd.to_numeric(df['Production(W)'], errors='coerce')

    # Scale data
    consumption = df['Consumption(W)'].values.reshape(-1, 1)
    production = df['Production(W)'].values.reshape(-1, 1)
    additional_features = df[['Hour', 'Day_of_Week', 'Month', 'Lag_1', 'Lag_60', 'Lag_1440', 'Lag_1_prod', 'Lag_60_prod', 'Lag_1440_prod', 'speed', 'dir', 'drybulb', 'cbl', 'soltot', 'rain']]
    additional_features = additional_features.apply(pd.to_numeric, errors='coerce')
    additional_features = additional_features.fillna(method='ffill').fillna(method='bfill')
    additional_features = additional_features.values
    split_point = int(len(consumption) * 0.80)

    scaler_consumption = RobustScaler()
    scaler_production = RobustScaler()
    scaler_features = RobustScaler()

    train_consumption = scaler_consumption.fit_transform(consumption[:split_point])
    test_consumption = scaler_consumption.transform(consumption[split_point:])
    train_production = scaler_production.fit_transform(production[:split_point])
    test_production = scaler_production.transform(production[split_point:])
    train_additional_features = scaler_features.fit_transform(additional_features[:split_point])
    test_additional_features = scaler_features.transform(additional_features[split_point:])

    window_size = 60
    batch_size = 64

    # Model for Consumption
    print("Training consumption model...")
    train_generator_consumption = prepare_timeseries_generator(train_consumption, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator_consumption = prepare_timeseries_generator(test_consumption, test_additional_features, window_size=window_size, batch_size=batch_size)

    input_shape = (window_size, train_generator_consumption[0][0].shape[2])
    consumption_model = create_bilstm_model(input_shape=input_shape)

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7)

    history_consumption = consumption_model.fit(train_generator_consumption, epochs=100, validation_data=test_generator_consumption, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)

    # Model for Production
    print("Training production model...")
    train_generator_production = prepare_timeseries_generator(train_production, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator_production = prepare_timeseries_generator(test_production, test_additional_features, window_size=window_size, batch_size=batch_size)

    production_model = create_bilstm_model_model(input_shape=input_shape)

    history_production = production_model.fit(train_generator_production, epochs=100, validation_data=test_generator_production, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)

    predictions_consumption = consumption_model.predict(test_generator_consumption)
    predictions_production = production_model.predict(test_generator_production)

    actual_consumption = scaler_consumption.inverse_transform(test_consumption[window_size:])
    actual_production = scaler_production.inverse_transform(test_production[window_size:])
    predicted_consumption = scaler_consumption.inverse_transform(predictions_consumption)
    predicted_production = scaler_production.inverse_transform(predictions_production)

    rmse_consumption = mean_squared_error(actual_consumption, predicted_consumption, squared=False)
    mae_consumption = mean_absolute_error(actual_consumption, predicted_consumption)
    r2_consumption = r2_score(actual_consumption, predicted_consumption)
    epsilon = 1e-5
    mape_consumption = np.mean(np.abs((actual_consumption - predictions_consumption) / np.maximum(np.abs(actual_consumption), epsilon))) * 100
    
    print(f"RMSE (Consumption): {rmse_consumption}")
    print(f"MAE (Consumption): {mae_consumption}")
    print(f"R² (Consumption): {r2_consumption}")
    print(f'MAPE (Consumption): {mape_consumption}%')

    rmse_production = mean_squared_error(actual_production, predicted_production, squared=False)
    mae_production = mean_absolute_error(actual_production, predicted_production)
    r2_production = r2_score(actual_production, predicted_production)
    mape_consumption = np.mean(np.abs((actual_production - predicted_production) / np.maximum(np.abs(actual_production), epsilon))) * 100

    print(f"RMSE (Production): {rmse_production}")
    print(f"MAE (Production): {mae_production}")
    print(f"R² (Production): {r2_production}")
    print(f'MAPE (Production): {mape_production}%')
    con_model_weights_path = f"{name}_2BiLSTM_Consumption_weights.h5"
    consumption_model.save_weights(con_model_weights_path)
    prod_model_weights_path = f"{name}_2BiLSTM_Production_weights.h5"
    production_model.save_weights(prod_model_weights_path)
    BiLSTM_models[name] = {
        'model': model,
        'actual_consumption': actual_consumption,
        'predicted_consumption': predicted_consumption,
        'rmse_consumption': rmse_production,
        'mae_consumption': mae_production,
        'r2_consumption': r2_production,
        'mape_consumption':  mape_consumption,
        'actual_production': actual_production,
        'predicted_production': predicted_production,
        'rmse_production': rmse_production,
        'mae_production': mae_production,
        'r2_production': r2_production,
        'mape_production':  mape_production,
        'con_model': consumption_model, 
        'prod_model':production_model,
        'con_model_weights_path': con_model_weights_path,
        'prod_model_weights_path': prod_model_weights_path
    }

    aligned_dates = df.index[split_point + window_size:split_point + window_size + len(actual_consumption)]

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual_consumption, label='Actual Consumption')
    plt.plot(aligned_dates, predicted_consumption, label='Predicted Consumption', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption (W)')
    plt.title(f'Actual vs Predicted Consumption for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2BiLSTM_House_{name}_full_consumption_results.jpeg", format='jpeg', dpi=300)  # Save the full plot for consumption
    plt.show()


    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual_production, label='Actual Production')
    plt.plot(aligned_dates, predicted_production, label='Predicted Production', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Production (W)')
    plt.title(f'Actual vs Predicted Production for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2BiLSTM_House_{name}_full_production_results.jpeg", format='jpeg', dpi=300)  # Save the full plot for production
    plt.show()


    points_in_24h = 1440 
    aligned_dates_24h = aligned_dates[:points_in_24h]

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual_consumption[:points_in_24h], label='Actual Consumption')
    plt.plot(aligned_dates_24h, predicted_consumption[:points_in_24h], label='Predicted Consumption', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption (W)')
    plt.title(f'Actual vs Predicted Consumption for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2BiLSTM_House_{name}_24h_consumption_results.jpeg", format='jpeg', dpi=300)  # Save the 24-hour plot for consumption
    plt.show()

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual_production[:points_in_24h], label='Actual Production')
    plt.plot(aligned_dates_24h, predicted_production[:points_in_24h], label='Predicted Production', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Production (W)')
    plt.title(f'Actual vs Predicted Production for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2BiLSTM_House_{name}_24h_production_results.jpeg", format='jpeg', dpi=300)  # Save the 24-hour plot for production
    plt.show()

    results_df = pd.DataFrame({
        'Date': aligned_dates,
        'Actual_Consumption': actual_consumption.flatten(), 
        'Predicted_Consumption': predicted_consumption.flatten(),
        'Actual_Production': actual_production.flatten(),
        'Predicted_Production': predicted_production.flatten()
    })

    results_df.to_csv(f"2BiLSTM_{name}_results.csv", index=False)
    print(f"Results saved to 2BiLSTM_{name}_results.csv")
    

In [ ]:
GRU_models = {}
for name, df in houses.items():
    print(f"Processing house: {name}")

    df = create_features(df)

    # Convert columns to numeric
    df['Consumption(W)'] = pd.to_numeric(df['Consumption(W)'], errors='coerce')
    df['Production(W)'] = pd.to_numeric(df['Production(W)'], errors='coerce')

    # Scale data
    consumption = df['Consumption(W)'].values.reshape(-1, 1)
    production = df['Production(W)'].values.reshape(-1, 1)
    additional_features = df[['Hour', 'Day_of_Week', 'Month', 'Lag_1', 'Lag_60', 'Lag_1440', 'Lag_1_prod', 'Lag_60_prod', 'Lag_1440_prod', 'speed', 'dir', 'drybulb', 'cbl', 'soltot', 'rain']]
    additional_features = additional_features.apply(pd.to_numeric, errors='coerce')
    additional_features = additional_features.fillna(method='ffill').fillna(method='bfill')
    additional_features = additional_features.values
    split_point = int(len(consumption) * 0.80)

    scaler_consumption = RobustScaler()
    scaler_production = RobustScaler()
    scaler_features = RobustScaler()

    train_consumption = scaler_consumption.fit_transform(consumption[:split_point])
    test_consumption = scaler_consumption.transform(consumption[split_point:])
    train_production = scaler_production.fit_transform(production[:split_point])
    test_production = scaler_production.transform(production[split_point:])
    train_additional_features = scaler_features.fit_transform(additional_features[:split_point])
    test_additional_features = scaler_features.transform(additional_features[split_point:])

    window_size = 60
    batch_size = 64

    # Model for Consumption
    print("Training consumption model...")
    train_generator_consumption = prepare_timeseries_generator(train_consumption, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator_consumption = prepare_timeseries_generator(test_consumption, test_additional_features, window_size=window_size, batch_size=batch_size)

    input_shape = (window_size, train_generator_consumption[0][0].shape[2])
    consumption_model = create_gru_model(input_shape=input_shape)

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7)

    history_consumption = consumption_model.fit(train_generator_consumption, epochs=100, validation_data=test_generator_consumption, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)

    # Model for Production
    print("Training production model...")
    train_generator_production = prepare_timeseries_generator(train_production, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator_production = prepare_timeseries_generator(test_production, test_additional_features, window_size=window_size, batch_size=batch_size)

    production_model = create_gru_model_model(input_shape=input_shape)

    history_production = production_model.fit(train_generator_production, epochs=100, validation_data=test_generator_production, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)

    predictions_consumption = consumption_model.predict(test_generator_consumption)
    predictions_production = production_model.predict(test_generator_production)

    actual_consumption = scaler_consumption.inverse_transform(test_consumption[window_size:])
    actual_production = scaler_production.inverse_transform(test_production[window_size:])
    predicted_consumption = scaler_consumption.inverse_transform(predictions_consumption)
    predicted_production = scaler_production.inverse_transform(predictions_production)

    rmse_consumption = mean_squared_error(actual_consumption, predicted_consumption, squared=False)
    mae_consumption = mean_absolute_error(actual_consumption, predicted_consumption)
    r2_consumption = r2_score(actual_consumption, predicted_consumption)
    epsilon = 1e-5
    mape_consumption = np.mean(np.abs((actual_consumption - predictions_consumption) / np.maximum(np.abs(actual_consumption), epsilon))) * 100
    
    print(f"RMSE (Consumption): {rmse_consumption}")
    print(f"MAE (Consumption): {mae_consumption}")
    print(f"R² (Consumption): {r2_consumption}")
    print(f'MAPE (Consumption): {mape_consumption}%')

    rmse_production = mean_squared_error(actual_production, predicted_production, squared=False)
    mae_production = mean_absolute_error(actual_production, predicted_production)
    r2_production = r2_score(actual_production, predicted_production)
    mape_consumption = np.mean(np.abs((actual_production - predicted_production) / np.maximum(np.abs(actual_production), epsilon))) * 100

    print(f"RMSE (Production): {rmse_production}")
    print(f"MAE (Production): {mae_production}")
    print(f"R² (Production): {r2_production}")
    print(f'MAPE (Production): {mape_production}%')
    con_model_weights_path = f"{name}_2GRU_Consumption_weights.h5"
    consumption_model.save_weights(con_model_weights_path)
    prod_model_weights_path = f"{name}_2GRU_Production_weights.h5"
    production_model.save_weights(prod_model_weights_path)
    GRU_models[name] = {
        'model': model,
        'actual_consumption': actual_consumption,
        'predicted_consumption': predicted_consumption,
        'rmse_consumption': rmse_production,
        'mae_consumption': mae_production,
        'r2_consumption': r2_production,
        'mape_consumption':  mape_consumption,
        'actual_production': actual_production,
        'predicted_production': predicted_production,
        'rmse_production': rmse_production,
        'mae_production': mae_production,
        'r2_production': r2_production,
        'mape_production':  mape_production,
        'con_model': consumption_model, 
        'prod_model':production_model,
        'con_model_weights_path': con_model_weights_path,
        'prod_model_weights_path': prod_model_weights_path
    }

    aligned_dates = df.index[split_point + window_size:split_point + window_size + len(actual_consumption)]

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual_consumption, label='Actual Consumption')
    plt.plot(aligned_dates, predicted_consumption, label='Predicted Consumption', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption (W)')
    plt.title(f'Actual vs Predicted Consumption for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2GRU_House_{name}_full_consumption_results.jpeg", format='jpeg', dpi=300)  # Save the full plot for consumption
    plt.show()


    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual_production, label='Actual Production')
    plt.plot(aligned_dates, predicted_production, label='Predicted Production', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Production (W)')
    plt.title(f'Actual vs Predicted Production for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2GRU_House_{name}_full_production_results.jpeg", format='jpeg', dpi=300)  # Save the full plot for production
    plt.show()


    points_in_24h = 1440 
    aligned_dates_24h = aligned_dates[:points_in_24h]

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual_consumption[:points_in_24h], label='Actual Consumption')
    plt.plot(aligned_dates_24h, predicted_consumption[:points_in_24h], label='Predicted Consumption', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption (W)')
    plt.title(f'Actual vs Predicted Consumption for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2GRU_House_{name}_24h_consumption_results.jpeg", format='jpeg', dpi=300)  # Save the 24-hour plot for consumption
    plt.show()

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual_production[:points_in_24h], label='Actual Production')
    plt.plot(aligned_dates_24h, predicted_production[:points_in_24h], label='Predicted Production', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Production (W)')
    plt.title(f'Actual vs Predicted Production for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2GRU_House_{name}_24h_production_results.jpeg", format='jpeg', dpi=300)  # Save the 24-hour plot for production
    plt.show()

    results_df = pd.DataFrame({
        'Date': aligned_dates,
        'Actual_Consumption': actual_consumption.flatten(), 
        'Predicted_Consumption': predicted_consumption.flatten(),
        'Actual_Production': actual_production.flatten(),
        'Predicted_Production': predicted_production.flatten()
    })

    results_df.to_csv(f"2GRU_{name}_results.csv", index=False)
    print(f"Results saved to 2GRU_{name}_results.csv")
    

In [ ]:
Transformer_models = {}
for name, df in houses.items():
    print(f"Processing house: {name}")

    df = create_features(df)

    # Convert columns to numeric
    df['Consumption(W)'] = pd.to_numeric(df['Consumption(W)'], errors='coerce')
    df['Production(W)'] = pd.to_numeric(df['Production(W)'], errors='coerce')

    # Scale data
    consumption = df['Consumption(W)'].values.reshape(-1, 1)
    production = df['Production(W)'].values.reshape(-1, 1)
    additional_features = df[['Hour', 'Day_of_Week', 'Month', 'Lag_1', 'Lag_60', 'Lag_1440', 'Lag_1_prod', 'Lag_60_prod', 'Lag_1440_prod', 'speed', 'dir', 'drybulb', 'cbl', 'soltot', 'rain']]
    additional_features = additional_features.apply(pd.to_numeric, errors='coerce')
    additional_features = additional_features.fillna(method='ffill').fillna(method='bfill')
    additional_features = additional_features.values
    split_point = int(len(consumption) * 0.80)

    scaler_consumption = RobustScaler()
    scaler_production = RobustScaler()
    scaler_features = RobustScaler()

    train_consumption = scaler_consumption.fit_transform(consumption[:split_point])
    test_consumption = scaler_consumption.transform(consumption[split_point:])
    train_production = scaler_production.fit_transform(production[:split_point])
    test_production = scaler_production.transform(production[split_point:])
    train_additional_features = scaler_features.fit_transform(additional_features[:split_point])
    test_additional_features = scaler_features.transform(additional_features[split_point:])

    window_size = 60
    batch_size = 64

    # Model for Consumption
    print("Training consumption model...")
    train_generator_consumption = prepare_timeseries_generator(train_consumption, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator_consumption = prepare_timeseries_generator(test_consumption, test_additional_features, window_size=window_size, batch_size=batch_size)

    input_shape = (window_size, train_generator_consumption[0][0].shape[2])
    consumption_model = cnn_lstm_attention_transformer_model(input_shape=input_shape, dropout=0.2, weight_decay=1e-7)

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7)

    history_consumption = consumption_model.fit(train_generator_consumption, epochs=100, validation_data=test_generator_consumption, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)

    # Model for Production
    print("Training production model...")
    train_generator_production = prepare_timeseries_generator(train_production, train_additional_features, window_size=window_size, batch_size=batch_size)
    test_generator_production = prepare_timeseries_generator(test_production, test_additional_features, window_size=window_size, batch_size=batch_size)

    production_model = cnn_lstm_attention_transformer_model(input_shape=input_shape, dropout=0.2, weight_decay=1e-7)

    history_production = production_model.fit(train_generator_production, epochs=100, validation_data=test_generator_production, callbacks=[early_stopping, reduce_lr], batch_size=batch_size)

    predictions_consumption = consumption_model.predict(test_generator_consumption)
    predictions_production = production_model.predict(test_generator_production)

    actual_consumption = scaler_consumption.inverse_transform(test_consumption[window_size:])
    actual_production = scaler_production.inverse_transform(test_production[window_size:])
    predicted_consumption = scaler_consumption.inverse_transform(predictions_consumption)
    predicted_production = scaler_production.inverse_transform(predictions_production)

    rmse_consumption = mean_squared_error(actual_consumption, predicted_consumption, squared=False)
    mae_consumption = mean_absolute_error(actual_consumption, predicted_consumption)
    r2_consumption = r2_score(actual_consumption, predicted_consumption)
    epsilon = 1e-5
    mape_consumption = np.mean(np.abs((actual_consumption - predictions_consumption) / np.maximum(np.abs(actual_consumption), epsilon))) * 100
    
    print(f"RMSE (Consumption): {rmse_consumption}")
    print(f"MAE (Consumption): {mae_consumption}")
    print(f"R² (Consumption): {r2_consumption}")
    print(f'MAPE (Consumption): {mape_consumption}%')

    rmse_production = mean_squared_error(actual_production, predicted_production, squared=False)
    mae_production = mean_absolute_error(actual_production, predicted_production)
    r2_production = r2_score(actual_production, predicted_production)
    mape_consumption = np.mean(np.abs((actual_production - predicted_production) / np.maximum(np.abs(actual_production), epsilon))) * 100

    print(f"RMSE (Production): {rmse_production}")
    print(f"MAE (Production): {mae_production}")
    print(f"R² (Production): {r2_production}")
    print(f'MAPE (Production): {mape_production}%')
    con_model_weights_path = f"{name}_2Hybrid_Consumption_weights.h5"
    consumption_model.save_weights(con_model_weights_path)
    prod_model_weights_path = f"{name}_2Hybrid_Production_weights.h5"
    production_model.save_weights(prod_model_weights_path)
    Transformer_models[name] = {
        'model': model,
        'actual_consumption': actual_consumption,
        'predicted_consumption': predicted_consumption,
        'rmse_consumption': rmse_production,
        'mae_consumption': mae_production,
        'r2_consumption': r2_production,
        'mape_consumption':  mape_consumption,
        'actual_production': actual_production,
        'predicted_production': predicted_production,
        'rmse_production': rmse_production,
        'mae_production': mae_production,
        'r2_production': r2_production,
        'mape_production':  mape_production,
        'con_model': consumption_model, 
        'prod_model':production_model,
        'con_model_weights_path': con_model_weights_path,
        'prod_model_weights_path': prod_model_weights_path
    }

    aligned_dates = df.index[split_point + window_size:split_point + window_size + len(actual_consumption)]

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual_consumption, label='Actual Consumption')
    plt.plot(aligned_dates, predicted_consumption, label='Predicted Consumption', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption (W)')
    plt.title(f'Actual vs Predicted Consumption for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2Hybrid_House_{name}_full_consumption_results.jpeg", format='jpeg', dpi=300)  # Save the full plot for consumption
    plt.show()


    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates, actual_production, label='Actual Production')
    plt.plot(aligned_dates, predicted_production, label='Predicted Production', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Production (W)')
    plt.title(f'Actual vs Predicted Production for {name}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2Hybrid_House_{name}_full_production_results.jpeg", format='jpeg', dpi=300)  # Save the full plot for production
    plt.show()


    points_in_24h = 1440 
    aligned_dates_24h = aligned_dates[:points_in_24h]

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual_consumption[:points_in_24h], label='Actual Consumption')
    plt.plot(aligned_dates_24h, predicted_consumption[:points_in_24h], label='Predicted Consumption', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Consumption (W)')
    plt.title(f'Actual vs Predicted Consumption for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2Hybrid_House_{name}_24h_consumption_results.jpeg", format='jpeg', dpi=300)  # Save the 24-hour plot for consumption
    plt.show()

    plt.figure(figsize=(20, 10))
    plt.plot(aligned_dates_24h, actual_production[:points_in_24h], label='Actual Production')
    plt.plot(aligned_dates_24h, predicted_production[:points_in_24h], label='Predicted Production', linestyle='--')
    plt.xlabel('Time')
    plt.ylabel('Energy Production (W)')
    plt.title(f'Actual vs Predicted Production for {name} - First 24 Hours')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"2Hybrid_House_{name}_24h_production_results.jpeg", format='jpeg', dpi=300)  # Save the 24-hour plot for production
    plt.show()

    results_df = pd.DataFrame({
        'Date': aligned_dates,
        'Actual_Consumption': actual_consumption.flatten(), 
        'Predicted_Consumption': predicted_consumption.flatten(),
        'Actual_Production': actual_production.flatten(),
        'Predicted_Production': predicted_production.flatten()
    })

    results_df.to_csv(f"2Hybrid_{name}_results.csv", index=False)
    print(f"Results saved to 2Hybrid_{name}_results.csv")
    